# 第 3 章 有記憶的簡易聊天程式–串接記錄與串流回應

## 3-1 文字形式的簡易聊天程式

先匯入必要的模組並建立客戶端：

In [ ]:
from google.colab import userdata
from IPython.display import display, Markdown
from rich.pretty import pprint
import sys
import openai

In [ ]:
client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

### 建立輔助函式與聊天程式雛形

In [ ]:
def get_reply_text(msg):
    try:
        response = client.responses.create(
            instructions='使用繁體中文',
            model="gpt-4.1-nano",
            input=msg
        )
        return response.output_text
    except openai.APIError as err:
        print(f'Error:{err.body["message"]}', file=sys.stderr)
        return ''

In [ ]:
print("直接按 ↵ 可結束對話")
while True:
    msg = input(">>> ")
    if not msg.strip(): break # 直接 ↵ 就結束
    reply = get_reply_text(msg)
    display(Markdown(f'{reply}'))

## 3-2 串接對話記錄

### 使用回應識別碼提供對談內容

In [ ]:
response = client.responses.create(
    instructions='使用繁體中文',
    model="gpt-4.1-nano",
    input="你知道什麼是流冰嗎？",
)
display(Markdown(response.output_text))

In [ ]:
response1 = client.responses.create(
    instructions='使用繁體中文',
    model="gpt-4.1-nano",
    input="哪裡看得到？",
)
display(Markdown(response1.output_text))

In [ ]:
response2 = client.responses.create(
    instructions='使用繁體中文',
    model="gpt-4.1-nano",
    input="哪裡看得到？",
    # 串接訊息
    previous_response_id=response.id,
)
display(Markdown(response2.output_text))

In [ ]:
pprint(response1.usage)
pprint(response2.usage)

In [ ]:
print(response.id)
print(response1.previous_response_id)
print(response2.previous_response_id)

#### 強制不儲存回應

In [ ]:
response = client.responses.create(
    instructions='使用繁體中文',
    model="gpt-4.1-nano",
    input="你知道什麼是流冰嗎？",
    store=False # 控制是否儲存回應
)
display(Markdown(response.output_text))

In [ ]:
response = client.responses.create(
    instructions='使用繁體中文',
    model="gpt-4.1-nano",
    input="哪裡看得到？",
    # 串接沒儲存的訊息會出錯
    previous_response_id=response.id,
)

#### 控制串接的輸入內容

### 幫聊天輔助函式串接對話過程

In [ ]:
class Chat:
    def __init__(self, client):
        self._last_id = None   # 紀錄最後回憶的識別碼
        self._client = client  # 叫用 API 的用戶端

    def get_reply_text(self, msg, **kwargs) -> str:
        instructions = kwargs.pop(
            'instructions', '使用繁體中文'
        )
        model = kwargs.pop('model', 'gpt-4.1-nano')
        try:
            response = self._client.responses.create(
                instructions=instructions,
                model=model,
                input=msg,
                previous_response_id=self._last_id, # 串接回應
                **kwargs
            )
            self._last_id = response.id # 更新回應識別碼
            return response.output_text
        except openai.APIError as err:
            print(
                f'Error:{err.body["message"]}',
                file=sys.stderr
            )
            return ''

    def loop(self, **kwargs) -> None:
        print("直接按 ↵ 可結束對話")
        while True:
            user_msg = input(">>> ")
            if not user_msg.strip(): break # 直接 ↵ 就結束
            reply = self.get_reply_text(user_msg, **kwargs)
            display(Markdown(f'{reply}'))

In [ ]:
Chat(client).loop()

## 3-3 使用串流功能即時顯示回覆內容

### 啟用串流功能

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="你好",
    stream=True,
)

In [ ]:
for event in response:
    pprint(event)

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="什麼是 pair programming？",
    stream=True,
)

text = ''
display_handle = display(text, display_id=True)
for event in response:
    # 篩選事件種類即可取得即時生成的部分內容
    if event.type == 'response.output_text.delta':
        text += event.delta
        display_handle.update(Markdown(text))

In [ ]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input="什麼是 peer review？",
    stream=True,
)

for event in response:
    # 最後的事件會有完整的生成內容，不用自己一段一段拼接
    if event.type == 'response.completed':
        display(Markdown('-' * 20))
        display(Markdown(
            f'{event.response.output_text}'
        ))
        display(Markdown('-' * 20))
        display(Markdown(
            f'{event.response.output[0].content[0].text}'
        ))

### 加入串流選項的聊天類別

In [ ]:
class Chat:
    def __init__(self, client, last_id=None):
        self._last_id = last_id
        self._client = client

    def get_reply_text(self, msg, **kwargs) -> str:
        instructions = kwargs.pop(
            'instructions', '使用繁體中文'
        )
        model = kwargs.pop('model', 'gpt-4.1-nano')
        stream = kwargs.pop('stream', False)
        try:
            response = self._client.responses.create(
                instructions=instructions,
                model=model,
                input=msg,
                stream=True, # 都以串流方式處理，簡化程式邏輯
                previous_response_id=self._last_id, # 串接回應
                **kwargs
            )
            for event in response:
                if event.type == 'response.output_text.delta':
                    if stream: # 串流模式生成片段內容
                        yield event.delta
                elif event.type == 'response.completed':
                    # 記錄識別碼
                    self._last_id = event.response.id
                    if not stream: # 非串流生成完整內容
                        yield event.response.output_text
        except openai.APIError as err:
            print(
                f'Error:{err.body["message"]}',
                file=sys.stderr
            )
            return ''

    def loop(self, **kwargs) -> None:
        print("直接按 ↵ 可結束對話")
        while True:
            user_msg = input(">>> ")
            if not user_msg.strip(): break # 直接 ↵ 就結束
            text = ''
            display_handle = display(text, display_id=True)
            for reply in self.get_reply_text(
                user_msg,
                **kwargs
            ):
                text += reply
                display_handle.update(Markdown(text))

In [ ]:
Chat(client).loop()

In [ ]:
chat1 = Chat(client)
chat1.loop(stream=True)

In [ ]:
chat2 = Chat(client, last_id=chat1._last_id)
chat2.loop(stream=True)

## 3-4 具有記憶的聊天程式

### 將回應識別碼儲存到檔案以及從檔案讀回的方法

In [ ]:
class Chat:
    def __init__(self, client, last_id=None):
        self._last_id = last_id
        self._client = client

    def get_reply_text(self, msg, **kwargs) -> str:
        instructions = kwargs.pop('instructions', '使用繁體中文')
        model = kwargs.pop('model', 'gpt-4.1-nano')
        stream = kwargs.pop('stream', False)
        try:
            response = self._client.responses.create(
                instructions=instructions,
                model=model,
                input=msg,
                stream=True, # 都以串流方式處理，簡化程式邏輯
                previous_response_id=self._last_id, # 串接回應
                **kwargs
            )
            for event in response:
                if event.type == 'response.output_text.delta':
                    if stream: # 串流模式生成片段內容
                        yield event.delta
                elif event.type == 'response.completed':
                    self._last_id = event.response.id # 記錄識別碼
                    if not stream: # 非串流生成完整內容
                        yield event.response.output_text
        except openai.APIError as err:
            print(f'Error:{err.body["message"]}', file=sys.stderr)
            return ''

    def loop(self, **kwargs) -> None:
        print("直接按 ↵ 可結束對話")
        while True:
            user_msg = input(">>> ")
            if not user_msg.strip(): break # 直接 ↵ 就結束
            text = ''
            display_handle = display(text, display_id=True)
            for reply in self.get_reply_text(user_msg, **kwargs):
                text += reply
                display_handle.update(Markdown(text))

    def save(self, filename) -> None:
        with open(filename, 'w') as f:
            f.write(self._last_id)

    def load(self, filename) -> None:
        with open(filename, 'r') as f:
            self._last_id = f.read()

### 可延續討論串交談的應用程式

In [ ]:
chat1 = Chat(client)
chat1.loop(stream=True)

In [ ]:
chat1.save('last_id')

In [ ]:
chat2 = Chat(client)
chat2.load('last_id')
chat2.loop(stream=True)

### 透過網頁檢視儲存的回應

你也可到 [dashboard 頁面](https://platform.openai.com/logs)查看對話記錄

### 利用程式碼管理對談記錄

#### 依據識別碼取得回應與輸入內容

In [ ]:
# 取得指定識別碼的回應
response = client.responses.retrieve(chat2._last_id)
print(response.output_text)

In [ ]:
# 取得指定識別碼回應的輸入（含對話串）
response = client.responses.input_items.list(
    chat2._last_id
)
pprint(response)

#### 顯示完整討論串

In [ ]:
# 顯示完整對話
inputs = client.responses.input_items.list(chat2._last_id)
response = client.responses.retrieve(chat2._last_id)
for item in inputs.data[::-1]:
    prompt = ">>> " if item.role == 'user' else ''
    for content in item.content:
        print(f'{prompt}{content.text}')
print(response.output_text)

#### 刪除討論串

In [ ]:
last_id = chat2._last_id
while last_id:
    response = client.responses.retrieve(last_id)
    last_id, curr_id = response.previous_response_id, last_id
    client.responses.delete(curr_id)

#### 幫聊天程式加上討論串管理功能

In [ ]:
class Chat:
    def __init__(self, client, last_id=None):
        self._last_id = last_id
        self._client = client

    def get_reply_text(self, msg, **kwargs) -> str:
        instructions = kwargs.pop('instructions', '使用繁體中文')
        model = kwargs.pop('model', 'gpt-4.1-nano')
        stream = kwargs.pop('stream', False)
        try:
            response = self._client.responses.create(
                instructions=instructions,
                model=model,
                input=msg,
                stream=True, # 都以串流方式處理，簡化程式邏輯
                previous_response_id=self._last_id, # 串接回應
                **kwargs
            )
            for event in response:
                if event.type == 'response.output_text.delta':
                    if stream: # 串流模式生成片段內容
                        yield event.delta
                elif event.type == 'response.completed':
                    self._last_id = event.response.id # 記錄識別碼
                    if not stream: # 非串流生成完整內容
                        yield event.response.output_text
        except openai.APIError as err:
            print(f'Error:{err.body["message"]}', file=sys.stderr)
            return ''

    def loop(self, **kwargs) -> None:
        print("直接按 ↵ 可結束對話")
        while True:
            user_msg = input(">>> ")
            if not user_msg.strip(): break # 直接 ↵ 就結束
            text = ''
            display_handle = display(text, display_id=True)
            for reply in self.get_reply_text(user_msg, **kwargs):
                text += reply
                display_handle.update(Markdown(text))

    def save(self, filename) -> None:
        with open(filename, 'w') as f:
            f.write(self._last_id)

    def load(self, filename) -> None:
        with open(filename, 'r') as f:
            self._last_id = f.read()

    def show_thread(self):
        if not self._last_id: return
        inputs = client.responses.input_items.list(
            self._last_id
        )
        response = client.responses.retrieve(self._last_id)
        for item in inputs.data[::-1]:
            prompt = ">>> " if item.role == 'user' else ''
            for content in item.content:
                print(f'{prompt}{content.text}')
        print(response.output_text)

    def delete_thread(self):
        if not self._last_id: return
        last_id = self._last_id
        while last_id:
            response = client.responses.retrieve(last_id)
            last_id, curr_id = (
                response.previous_response_id,
                last_id
            )
            client.responses.delete(curr_id)
        self._last_id = None

In [ ]:
chat = Chat(client)
chat.loop(stream=True)

In [ ]:
chat.show_thread()

In [ ]:
chat.delete_thread()

## 3-5 手動建立對話記錄

### 自行建立對話記錄

In [ ]:
class ChatWithHistList:
    def __init__(self, client):
        self._hist = [] # 記錄對話過程
        self._client = client

    def get_reply_text(self, msg, **kwargs) -> str:
        instructions = kwargs.pop(
            'instructions', '使用繁體中文'
        )
        model = kwargs.pop('model', 'gpt-4.1-nano')
        stream = kwargs.pop('stream', False)
        try:
            response = self._client.responses.create(
                instructions=instructions,
                model=model,
                input=self._hist + [{
                    'role': 'user', 'content': msg
                }],
                stream=True, # 都以串流方式處理，簡化程式邏輯
                store=False, # 不儲存對話
                **kwargs
            )
            for event in response:
                if event.type == 'response.output_text.delta':
                    if stream: # 串流模式生成片段內容
                        yield event.delta
                elif event.type == 'response.completed':
                    # 新增對話
                    self._hist += [
                        {'role': 'user', 'content': msg},
                        {'role': 'assistant',
                         'content': event.response.output_text}
                    ]
                    if not stream: # 非串流生成完整內容
                        yield event.response.output_text
        except openai.APIError as err:
            print(
                f'Error:{err.body["message"]}',
                file=sys.stderr
            )
            return ''

    def loop(self, **kwargs) -> None:
        print("直接按 ↵ 可結束對話")
        while True:
            user_msg = input(">>> ")
            if not user_msg.strip(): break # 直接 ↵ 就結束
            text = ''
            display_handle = display(text, display_id=True)
            for reply in self.get_reply_text(user_msg, **kwargs):
                text += reply
                display_handle.update(Markdown(text))

In [ ]:
ChatWithHistList(client).loop(stream=True)

### 儲存對話記錄

In [ ]:
import pickle

In [ ]:
class ChatWithHistList:
    def __init__(self, client):
        self._hist = [] # 記錄對話過程
        self._client = client

    def get_reply_text(self, msg, **kwargs) -> str:
        instructions = kwargs.pop('instructions', '使用繁體中文')
        model = kwargs.pop('model', 'gpt-4.1-nano')
        stream = kwargs.pop('stream', False)
        try:
            response = self._client.responses.create(
                instructions=instructions,
                model=model,
                input=self._hist + [{'role': 'user', 'content': msg}],
                stream=True, # 都以串流方式處理，簡化程式邏輯
                store=False, # 不儲存對話
                **kwargs
            )
            for event in response:
                if event.type == 'response.output_text.delta':
                    if stream: # 串流模式生成片段內容
                        yield event.delta
                elif event.type == 'response.completed':
                    # 新增對話
                    self._hist += [
                        {'role': 'user', 'content': msg},
                        {'role': 'assistant',
                         'content': event.response.output_text}
                    ]
                    if not stream: # 非串流生成完整內容
                        yield event.response.output_text
        except openai.APIError as err:
            print(f'Error:{err.body["message"]}', file=sys.stderr)
            return ''

    def loop(self, **kwargs) -> None:
        print("直接按 ↵ 可結束對話")
        while True:
            user_msg = input(">>> ")
            if not user_msg.strip(): break # 直接 ↵ 就結束
            text = ''
            display_handle = display(text, display_id=True)
            for reply in self.get_reply_text(user_msg, **kwargs):
                text += reply
                display_handle.update(Markdown(text))

    def save(self, filename) -> None:
        with open(filename, 'wb') as f:
            pickle.dump(self._hist, f)

    def load(self, filename) -> None:
        with open(filename, 'rb') as f:
            self._hist = pickle.load(f)

In [ ]:
chat1 = ChatWithHistList(client)
chat1.loop(stream=True)

In [ ]:
chat1.save('hist.db')

In [ ]:
chat2 = ChatWithHistList(client)
chat2.load('hist.db')
chat2.loop(stream=True)